# General pipeline for project 1
This is an example pipeline showing you how to  
(1) Load the provided data;  
(2) Train models on the train set, and use the validation set to evaluate your model performance;  
(3) Generate predictions (pred.csv) on the test set, which is ready for submission.

In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
import torch
torch.cuda.is_available()

False

### (1) Loading data
The following code shows how to load the datasets for this project.  
Among which, we do not release the labels (the "stars" column) for the test set. You may evaluate your trained model on the validation set instead.

However, your submitted predictions (``pred.csv``) should be generated on the test set.

In [ ]:
def load_data(split_name='train', columns=['text', 'stars'], folder='data'):
    '''
        "split_name" may be set as 'train', 'valid' or 'test' to load the corresponding dataset.
        
        You may also specify the column names to load any columns in the .csv data file.
        Among many, "text" can be used as model input, and "stars" column is the labels (sentiment). 
        If you like, you are free to use columns other than "text" for prediction.
    '''
    try:
        print(f"select [{', '.join(columns)}] columns from the {split_name} split")
        df = pd.read_csv(f'{folder}/{split_name}.csv')
        df = df.loc[:,columns]
        print("Success")
        return df
    except:
        print(f"Failed loading specified columns... Returning all columns from the {split_name} split")
        df = pd.read_csv(f'{folder}/{split_name}.csv')
        return df

In [ ]:
train_df = load_data('train', columns=['text', 'stars'])
valid_df = load_data('valid', columns=['text', 'stars'])
# the test set labels (the 'stars' column) are not available! So the following code will instead return all columns
test_df = load_data('test', columns=['text', 'stars'])

select [text, stars] columns from the train split
Success
select [text, stars] columns from the valid split
Success
select [text, stars] columns from the test split
Failed loading specified columns... Returning all columns from the test split


In [ ]:
# test_df.columns
# print(train_df.columns)
# print(valid_df.columns)
# print(test_df.columns)
test_df

,business_id,cool,date,funny,review_id,text,useful,user_id
0,IKcZpSuELli7DUjU2fKGNg,1,2015-04-07 17:17:39,0,I77zZlSdCFAClxdjHwPcxw,OMG! I'm an avid spray tanner and have been al...,2,tUZtqzqE0bIOcLelcR4opg
1,vbVJzKDhHlhMnKRpES5QzQ,1,2017-06-30 17:42:40,0,ioFNKarf29KGjRZdH0qC8Q,Sets the standard. Authentic. Outstanding. Cou...,1,Gwvrebru-kDM1N51aeJiFg
2,GdPWJo3z4ySEXpF7Wkn3FA,0,2014-08-02 05:53:47,2,9429anmcYIcaEcMptJCNKQ,Came on 7/23/2014 with a group of 10 - service...,1,at7dS8gtLiEwd_4uHv231A
3,BYc5IFQPq-PLVXnYjDp6vw,0,2015-02-20 19:31:21,0,PsUCdt7PKjzgBC0c7xXhJA,I love Bobs Subs! Tasty n made to order...yum!...,0,vaQxpV8IXqRmCIAHovP4NA
4,Wxxvi3LZbHNIDwJ-ZimtnA,0,2012-06-27 00:44:08,0,GQBlykKyShQcNeu2ivLdSA,This is my hotel of choice on the strip. I re...,0,dy_4NAZ0KR2bDoB9qAOMRg
...,...,...,...,...,...,...,...,...
3995,nqKLJ9q7jvnzRKW4RIu3qw,0,2015-08-28,0,LJ7Fqb2hlrGhYL-HK_VGAQ,I have a glass wall and ceiling sunroom by Fou...,0,nuxXEP96hVyEjmZv2QZ1fQ
3996,Avr_qHllB60Ade8IwnU7Nw,0,2013-10-21 00:59:29,0,M1SmtK0Fy2GPA26yzrfGWg,I love Kohls. My favorite place to clothes sho...,0,nURWL_BFa3Mp90ZtmmGakQ
3997,PsYU8cEDE3oewzKCyVt5bw,0,2016-07-05 02:10:44,0,wSI7X7qqMfDQxkNFHkOYwA,"Wow, this was genuinely terrible food. I order...",1,9gX_LpIjPD3McOdKz8ixNQ
3998,4_oBvzWuJe3_Y8PRUaPmug,0,2015-09-26 18:32:56,0,DQjRGs939gBPqfoIqK7VYQ,Worst service ever! My clothes were tagged wro...,1,mPFcpFlSG4A964z7aPJchg


### (2) Training and validating 
The following example shows you how to train your model using the train set, and evaluate on the validation set.  
As an example, we only use the text data for training. Feel free to use other columns in your implementation.  

The model performance on the validation set can be roughly regarded as your models final performance, so we can use it to search for optimal hyper-parameters.

In [ ]:
# Prepare the data.
# As an example, we only use the text data. 
x_train = train_df['text']
y_train = train_df['stars']
  
x_valid = valid_df['text']
y_valid = valid_df['stars']

x_test = test_df['text']

 You can use the valid data to choose the hyperparameters.
As an example, you can decide which value of C (1 or 100) is better by evaluating on the valid data.

In [ ]:
# build the first linear model with TFIDF feature
tfidf = TfidfVectorizer()
lr1 = LogisticRegression(C=100)
steps = [('tfidf', tfidf),('lr', lr1)]
pipe1 = Pipeline(steps)

In [ ]:
# train the first model
pipe1.fit(x_train, y_train)

/home/jchengaj/miniconda3/envs/comp4332/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('lr', LogisticRegression(C=100))])

In [ ]:
# validate on the validation set
y_pred = pipe1.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

              precision    recall  f1-score   support

           1       0.70      0.72      0.71       282
           2       0.30      0.27      0.29       136
           3       0.37      0.40      0.38       212
           4       0.47      0.46      0.47       466
           5       0.77      0.77      0.77       904

    accuracy                           0.62      2000
   macro avg       0.52      0.52      0.52      2000
weighted avg       0.62      0.62      0.62      2000




[[202  35  23   7  15]
 [ 45  37  32  15   7]
 [ 19  32  85  56  20]
 [  8  14  62 216 166]
 [ 14   4  30 164 692]]
accuracy 0.616


In [ ]:
# build the second linear model with TFIDF feature
tfidf = TfidfVectorizer()
lr2 = LogisticRegression(C=1)
steps = [('tfidf', tfidf),('lr', lr2)]
pipe2 = Pipeline(steps)

In [ ]:
# train the second model
pipe2.fit(x_train, y_train)

/home/jchengaj/miniconda3/envs/comp4332/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()), ('lr', LogisticRegression(C=1))])

In [ ]:
# validate on the validation set
y_pred = pipe2.predict(x_valid)
print(classification_report(y_valid, y_pred))
print("\n\n")
print(confusion_matrix(y_valid, y_pred))
print('accuracy', np.mean(y_valid == y_pred))

              precision    recall  f1-score   support

           1       0.70      0.81      0.75       282
           2       0.35      0.18      0.23       136
           3       0.48      0.32      0.38       212
           4       0.51      0.46      0.48       466
           5       0.74      0.86      0.79       904

    accuracy                           0.65      2000
   macro avg       0.55      0.52      0.53      2000
weighted avg       0.63      0.65      0.63      2000




[[228  13   5  12  24]
 [ 50  24  36  13  13]
 [ 21  21  67  74  29]
 [ 18   6  24 214 204]
 [ 11   5   8 107 773]]
accuracy 0.653


 We find the second model (pipe2) has higher accuracy, then we use the second model to make predictions on test data. In practice, you may not only focus on the accuracy, but also other metrics (precision, recall, f1), since the label distribution is not always balanced.

### (3) Generate predictions on the test set

In [ ]:
predict_test = pipe2.predict(x_test)

In [ ]:
predict_test

array([5, 5, 1, ..., 1, 1, 2])

In [ ]:
# save your model predictions
pred_df = pd.DataFrame({'stars': predict_test, 'review_id': test_df['review_id']})
pred_df.to_csv('pred.csv', index=False)

 Then you may submit the predictions `pred.csv` on the test set. 